# Subsystem Template:

```java
class Subsystem extends SubsystemBase {

    // Hardware
    private final SparkMax leaderMotor; // Motor controllers, either SparkMax or SparkFlex, ask mechanical if unsure
    private final SparkMax followerMotor;

    private final SparkMaxConfig leaderConfig; // Motor configuration
    private final SparkMaxConfig followerConfig;

    private final RelativeEncoder encoder; // Motor's integrated encoder, resets when robot is turned off/on

    // Onboard PID
    private final SparkClosedLoopController pidController;

    // Constructor
    public ElevatorSubsystem() {

        // Initialize motors
        leaderMotor = new SparkMax(ElevatorConstants.ELEVATOR_LEADER_CAN_ID, ElevatorConstants.MOTOR_TYPE);
        followerMotor = new SparkMax(ElevatorConstants.ELEVATOR_FOLLOWER_CAN_ID, ElevatorConstants.MOTOR_TYPE);

        // Initialize motor configurations
        leaderConfig = new SparkMaxConfig();
        leaderConfig.idleMode(IdleMode.kBrake).
                            smartCurrentLimit(40).
                            closedLoop.p(ElevatorConstants.kP).i(ElevatorConstants.kI).d(ElevatorConstants.kD);

        followerConfig = new SparkMaxConfig();
        followerConfig.idleMode(IdleMode.kBrake)
                              .smartCurrentLimit(40)
                              .follow(elevatorLeaderMotor, true);

        // Apply Configurations
        leaderMotor.configure(elevatorLeaderConfig, ResetMode.kResetSafeParameters, PersistMode.kPersistParameters);
        followerMotor.configure(elevatorFollowerConfig, ResetMode.kResetSafeParameters, PersistMode.kPersistParameters);

        // Initialize Encoder and PID Controller
        encoder = elevatorLeaderMotor.getEncoder();
        pidController = elevatorLeaderMotor.getClosedLoopController();
    }

    // Move motor to desired position using PID control
    public void setSetpoint(double setpoint) {
        pidController.setReference(setpoint, ControlType.kPosition);
    }

    // Manual motor control
    public void setMotor(double speed) {
        // Safety feature to limit bounds of movement
        if ((speed < 0 && getEncoder() <= SubsystemConstants.LOWER_BOUND) ||
            (speed > 0 && getEncoder() >= SubsystemConstants.UPPER_BOUND)) {
            leaderMotor.set(0);
        } else {
            leaderMotor.set(speed);
        }
    }
    
}
```

---

---
### 1. The Class

```java
class Subsystem extends SubsystemBase {

}
```

Don't worry about the term *extends* here, simply replace *Subsystem* with your desired Subsystem name
```

### 2. Variables Inside the Class
```java
    private final SparkMax leaderMotor; // Motor controllers, either SparkMax or SparkFlex, ask mechanical if unsure
    private final SparkMax followerMotor;

    private final SparkMaxConfig leaderConfig; // Motor configuration
    private final SparkMaxConfig followerConfig;
```
Almost every subsystem will have at least one motor, this requires two things:
- A corresponding motor controller, we use SparkFlex and SparkMax with NEO motors (brushless)
- A motor configuration, these configurations allow us to set the limitations of our motors to avoid battery brownout and account for inverted controls

Here are some of the common configurations we use and what they do:
- idleMode(): Sets the motors to either coast or break when not powered, allowing us to manually move them or not
- smartCurrentLimit(): Limits the amount of power the battery can apply to the subsystem hopefully limiting brownouts
- closedLoop(): This is our PID control, where we can tune values to allow automatic positioning to become more precise
- follow(): How we designate a follower motor and invert its controlls if necessary

```java
leaderMotor.configure(elevatorLeaderConfig, ResetMode.kResetSafeParameters, PersistMode.kPersistParameters);
followerMotor.configure(elevatorFollowerConfig, ResetMode.kResetSafeParameters, PersistMode.kPersistParameters);
```
This is simply how we apply the configurations we have just come up with

### 3. Manual subsystem control
```java
public void setMotor(double speed) {
    // Safety feature to limit bounds of movement
    if ((speed < 0 && getEncoder() <= SubsystemConstants.LOWER_BOUND) ||
        (speed > 0 && getEncoder() >= SubsystemConstants.UPPER_BOUND)) {
        leaderMotor.set(0);
    } else {
        leaderMotor.set(speed);
    }
}
```

This method allows us to input a speed value directly to the motor between 0 and 1 (as a percent). The *if-else* statement acts as a safety feature to limit the movement of our subsystem and avoid breakage. LOWER_BOUND & UPPER_BOUND are encoder values we can set through trial and error.

### 3. Automatic subsystem control
```java
public void setSetpoint(double setpoint) {
    pidController.setReference(setpoint, ControlType.kPosition);
}
```
In this method we give an encoder value *setpoint* and let the PID controller we established earlier move our motors until that encoder position is reached, at which point it attempts to stabilize at that position.